DEPENDENCIAS

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import k_means 

LEEMOS LOS DATOS

In [30]:
datos = pd.read_excel("./inputs/Gerd_Eurostat.xlsx", header=0)
datos.columns = datos.columns.str.strip()
datos.head()

,"sectfund,unit,geo\time",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"ABR,PC_TOT,AT",20 e,19.4,18 e,18.4,17.9,16.4 e,16.8,16.1 e,16.9,15.7 e,16.6,15.2 ep
1,"ABR,PC_TOT,BA",:,:,:,:,:,:,:,:,:,48.7,53.9,:
2,"ABR,PC_TOT,BE",12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,:
3,"ABR,PC_TOT,BG",5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,:
4,"ABR,PC_TOT,CH",:,5.2,:,:,:,6,:,:,:,12.1,:,:


In [33]:
nuevas_cabeceras = "2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014".split(',')
columnas_actuales = list(datos.columns)
columnas_actuales[1:13] = nuevas_cabeceras
datos.columns = columnas_actuales
datos.head()

,"sectfund,unit,geo\time",2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,"ABR,PC_TOT,AT",20 e,19.4,18 e,18.4,17.9,16.4 e,16.8,16.1 e,16.9,15.7 e,16.6,15.2 ep
1,"ABR,PC_TOT,BA",:,:,:,:,:,:,:,:,:,48.7,53.9,:
2,"ABR,PC_TOT,BE",12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,:
3,"ABR,PC_TOT,BG",5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,:
4,"ABR,PC_TOT,CH",:,5.2,:,:,:,6,:,:,:,12.1,:,:


DIVIDIMOS LA COLUMNA 1

La columna tiene un tabulador en el nombre principal, por lo que lo renombramos y hacemos el split por comas de forma manual

In [34]:
print(datos.columns[0])

sectfund,unit,geo\time


In [35]:
# Renombrar la columna problemática
datos.rename(columns={datos.columns[0]: 'metadata'}, inplace=True)
print(datos.columns[0])

metadata


In [36]:
new_cols = "sectfund,unit,geo-time".split(',')
datos[new_cols] = datos['metadata'].str.split(',',expand=True)
datos.head()

,metadata,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,sectfund,unit,geo-time
0,"ABR,PC_TOT,AT",20 e,19.4,18 e,18.4,17.9,16.4 e,16.8,16.1 e,16.9,15.7 e,16.6,15.2 ep,ABR,PC_TOT,AT
1,"ABR,PC_TOT,BA",:,:,:,:,:,:,:,:,:,48.7,53.9,:,ABR,PC_TOT,BA
2,"ABR,PC_TOT,BE",12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,:,ABR,PC_TOT,BE
3,"ABR,PC_TOT,BG",5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,:,ABR,PC_TOT,BG
4,"ABR,PC_TOT,CH",:,5.2,:,:,:,6,:,:,:,12.1,:,:,ABR,PC_TOT,CH


In [41]:
np.unique(datos['sectfund'])

array(['ABR', 'BES', 'GOV', 'HES', 'PNP'], dtype=object)

In [42]:
np.unique(datos['unit'])

array(['PC_TOT'], dtype=object)

Como la col unit es un único elemento en todas las filas la podemos eliminar al no aportar valor

In [ ]:
datos = datos.drop(labels = "unit", axis=1)
#Tambien eliminamos la columna 1 que ya la tenemos dividida
datos = datos.drop(labels = "metadata", axis=1)

In [50]:
datos.head()

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,sectfund,geo-time
0,20 e,19.4,18 e,18.4,17.9,16.4 e,16.8,16.1 e,16.9,15.7 e,16.6,15.2 ep,ABR,AT
1,:,:,:,:,:,:,:,:,:,48.7,53.9,:,ABR,BA
2,12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,:,ABR,BE
3,5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,:,ABR,BG
4,:,5.2,:,:,:,6,:,:,:,12.1,:,:,ABR,CH


Sustituimos los : por Nan

In [58]:
print(datos.dtypes)

2003        object
2004        object
2005        object
2006        object
2007        object
2008        object
2009        object
2010        object
2011        object
2012        object
2013        object
2014        object
sectfund    object
geo-time    object
dtype: object


In [59]:
datos = datos.replace(r'^\s*:\s*$', np.nan, regex=True)

In [66]:
datos.head()

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,sectfund,geo-time
0,20 e,19.4,18 e,18.4,17.9,16.4 e,16.8,16.1 e,16.9,15.7 e,16.6,15.2 ep,ABR,AT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.7,53.9,NaN,ABR,BA
2,12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,NaN,ABR,BE
3,5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,NaN,ABR,BG
4,NaN,5.2,NaN,NaN,NaN,6,NaN,NaN,NaN,12.1,NaN,NaN,ABR,CH


Nos quedamos solo con los numeros

In [67]:
datos[datos.columns[:-2]] = datos[datos.columns[:-2]].replace(r'[^0-9\.\-]', '', regex=True)
datos.head()

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,sectfund,geo-time
0,20,19.4,18,18.4,17.9,16.4,16.8,16.1,16.9,15.7,16.6,15.2,ABR,AT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.7,53.9,NaN,ABR,BA
2,12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,NaN,ABR,BE
3,5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,NaN,ABR,BG
4,NaN,5.2,NaN,NaN,NaN,6,NaN,NaN,NaN,12.1,NaN,NaN,ABR,CH


In [73]:
datos[datos.columns[:-2]] = datos[datos.columns[:-2]].apply(pd.to_numeric, errors='coerce')

In [75]:
datos[datos.columns[:-2]] = datos[datos.columns[:-2]].astype(float)
datos.head()

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,sectfund,geo-time
0,20.0,19.4,18.0,18.4,17.9,16.4,16.8,16.1,16.9,15.7,16.6,15.2,ABR,AT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.7,53.9,NaN,ABR,BA
2,12.9,12.3,12.4,13.3,13.0,12.3,12.1,13.3,13.0,13.0,13.2,NaN,ABR,BE
3,5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,NaN,ABR,BG
4,NaN,5.2,NaN,NaN,NaN,6.0,NaN,NaN,NaN,12.1,NaN,NaN,ABR,CH


Cogemos la columna de 2013 que es la que más datos tiene

In [80]:
datos_clean = datos[["2013", "sectfund", "geo-time"]]
datos_clean.head()

,2013,sectfund,geo-time
0,16.6,ABR,AT
1,53.9,ABR,BA
2,13.2,ABR,BE
3,48.3,ABR,BG
4,NaN,ABR,CH


Tabla pivotada

In [81]:
pivot_2013 = datos.pivot(index="geo-time", columns="sectfund", values="2013")
pivot_2013

sectfund,ABR,BES,GOV,HES,PNP
geo-time,,,,,
AT,16.6,48.7,33.6,0.6,0.5
BA,53.9,16.4,25.9,3.7,0.0
BE,13.2,56.9,28.5,1.0,0.4
BG,48.3,19.5,31.6,0.1,0.5
CH,NaN,NaN,NaN,NaN,NaN
CN_X_HK,0.9,74.6,21.1,NaN,NaN
CY,19.6,12.1,62.1,5.6,0.6
CZ,27.2,37.6,34.7,0.5,0.1
DE,5.2,65.4,29.1,NaN,0.3


In [82]:
# Eliminamos CH y EEUU por no tener nada de info
pivot_clean_2013 = pivot_2013.drop(["CH","US"])
pivot_clean_2013

sectfund,ABR,BES,GOV,HES,PNP
geo-time,,,,,
AT,16.6,48.7,33.6,0.6,0.5
BA,53.9,16.4,25.9,3.7,0.0
BE,13.2,56.9,28.5,1.0,0.4
BG,48.3,19.5,31.6,0.1,0.5
CN_X_HK,0.9,74.6,21.1,NaN,NaN
CY,19.6,12.1,62.1,5.6,0.6
CZ,27.2,37.6,34.7,0.5,0.1
DE,5.2,65.4,29.1,NaN,0.3
DK,7.4,57.9,30.4,NaN,4.3


2. MODELIZACIÓN

In [85]:
pt_2013 = pivot_clean_2013.fillna(pivot_clean_2013.mean())

In [86]:
#Creamos los clusters con k_means
v = k_means(pt_2013, n_clusters = 5)

In [88]:
#Sacamos los 5 clusters
cl1 = v[0]==0
ind1 = np.where(cl1)
cl2 = v[0]==1
ind2 = np.where(cl2)
cl3 = v[0]==2
ind3 = np.where(cl3)
cl4 = v[0]==3
ind4 = np.where(cl4)
cl5 = v[0]==4
ind5 = np.where(cl5)

In [91]:
pt_2013.head()

sectfund,ABR,BES,GOV,HES,PNP
geo-time,,,,,
AT,16.6,48.7,33.6,0.600000,0.500000
BA,53.9,16.4,25.9,3.700000,0.000000
BE,13.2,56.9,28.5,1.000000,0.400000
BG,48.3,19.5,31.6,0.100000,0.500000
CN_X_HK,0.9,74.6,21.1,2.941667,0.992105


In [92]:
print(f"CLUSTER 1 {pt_2013.index[ind2[0]]}")

CLUSTER 1 Index([], dtype='object', name='geo-time')


In [96]:
pt_2013

sectfund,ABR,BES,GOV,HES,PNP
geo-time,,,,,
AT,16.6,48.7,33.6,0.600000,0.500000
BA,53.9,16.4,25.9,3.700000,0.000000
BE,13.2,56.9,28.5,1.000000,0.400000
BG,48.3,19.5,31.6,0.100000,0.500000
CN_X_HK,0.9,74.6,21.1,2.941667,0.992105
CY,19.6,12.1,62.1,5.600000,0.600000
CZ,27.2,37.6,34.7,0.500000,0.100000
DE,5.2,65.4,29.1,2.941667,0.300000
DK,7.4,57.9,30.4,2.941667,4.300000


OTRA FORMA DE CLUSTERIZAR

In [97]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Escalar los datos (opcional pero muy recomendable)
scaler = StandardScaler()
pt_2013_scaled = scaler.fit_transform(pt_2013)

# Aplicar KMeans
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(pt_2013_scaled)

# Obtener las etiquetas (a qué cluster pertenece cada fila)
labels = kmeans.labels_

# Mostrar los índices de cada cluster
for i in range(5):
    print(f"\nCLUSTER {i}:")
    print(pt_2013.index[labels == i])



CLUSTER 0:
Index(['CN_X_HK', 'DE', 'FI', 'JP', 'KR', 'TR'], dtype='object', name='geo-time')

CLUSTER 1:
Index(['AT', 'BE', 'CZ', 'EA19', 'EE', 'ES', 'EU28', 'FR', 'HR', 'HU', 'IE',
       'IS', 'ME', 'MT', 'NO', 'PL', 'PT', 'SI', 'SK'],
      dtype='object', name='geo-time')

CLUSTER 2:
Index(['RS'], dtype='object', name='geo-time')

CLUSTER 3:
Index(['DK', 'IT', 'NL', 'SE', 'UK'], dtype='object', name='geo-time')

CLUSTER 4:
Index(['BA', 'BG', 'CY', 'EL', 'LT', 'LU', 'LV', 'RO', 'RU'], dtype='object', name='geo-time')


In [99]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Escalar los datos
scaler = StandardScaler()
pt_2013_scaled = scaler.fit_transform(pt_2013)

# Aplicar KMeans
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(pt_2013_scaled)

# Obtener los centroides (en espacio escalado)
centroides_escalados = kmeans.cluster_centers_

# Si quieres verlos en el espacio original:
centroides_originales = scaler.inverse_transform(centroides_escalados)

# Mostrar los centroides como DataFrame para mejor visualización
import pandas as pd

df_centroides = pd.DataFrame(centroides_originales, columns=pt_2013.columns)
print(df_centroides)

for i in range(5):
    print(f"\nCLUSTER {i}:")
    print(pt_2013.index[labels == i])

sectfund        ABR        BES        GOV        HES           PNP
0          3.200000  66.816667  23.816667   5.513889  1.215351e+00
1         14.315789  47.068421  36.457895   1.807456  5.105263e-01
2          7.800000   7.500000  59.500000  25.100000 -1.110223e-16
3         10.940000  52.280000  32.500000   1.308333  3.580000e+00
4         30.588889  22.588889  44.288889   2.066667  5.324561e-01

CLUSTER 0:
Index(['CN_X_HK', 'DE', 'FI', 'JP', 'KR', 'TR'], dtype='object', name='geo-time')

CLUSTER 1:
Index(['AT', 'BE', 'CZ', 'EA19', 'EE', 'ES', 'EU28', 'FR', 'HR', 'HU', 'IE',
       'IS', 'ME', 'MT', 'NO', 'PL', 'PT', 'SI', 'SK'],
      dtype='object', name='geo-time')

CLUSTER 2:
Index(['RS'], dtype='object', name='geo-time')

CLUSTER 3:
Index(['DK', 'IT', 'NL', 'SE', 'UK'], dtype='object', name='geo-time')

CLUSTER 4:
Index(['BA', 'BG', 'CY', 'EL', 'LT', 'LU', 'LV', 'RO', 'RU'], dtype='object', name='geo-time')
